<a href="https://colab.research.google.com/github/nyp-sit/it3103/blob/main/week11/text_classification_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification using RNN
In this lab exercise, we will learn to use LSTM (an RNN variant) to train a model to classify a piece of text as expressing positive sentiment or negative sentiment.

## Setup

In [ ]:
import os
import shutil
import tensorflow as tf

from datetime import datetime
import tensorflow as tf

### Download the IMDb Dataset
You will use the [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/). You will train a sentiment classifier model on this dataset.

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

Take a look at the `train/` directory. It has `pos` and `neg` folders with movie reviews labelled as positive and negative respectively. You will use reviews from `pos` and `neg` folders to train a binary classification model.

In [ ]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

The `train` directory also has additional folders which should be removed before creating training dataset.

In [ ]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Next, create a `tf.data.Dataset` using `tf.keras.preprocessing.text_dataset_from_directory`. You can read more about this utility from the [api documentation](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text_dataset_from_directory). 

Use the `train` directory to create both train and validation datasets with a split of 20% for validation. Also note that here we use a smaller batch size of 128, as our model now is more complex, and will use up some significant memory, leaving little room for larger batch size.

In [ ]:
batch_size = 128
seed = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2, 
    subset='training', seed=seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2, 
    subset='validation', seed=seed)

Take a look at a few movie reviews and their labels `(1: positive, 0: negative)` from the train dataset.


In [ ]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(label_batch[i].numpy(), text_batch[i].numpy())

### Configure the dataset for performance

These are two important methods you should use when loading data to make sure that I/O does not become blocking.

`.cache()` keeps data in memory after it's loaded off disk. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache, which is more efficient to read than many small files.

`.prefetch()` overlaps data preprocessing and model execution while training. 

You can learn more about both methods, as well as how to cache data to disk in the [data performance guide](https://www.tensorflow.org/guide/data_performance).

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Text preprocessing

Next, define the dataset preprocessing steps required for your sentiment classification model. Initialize a TextVectorization layer with the desired parameters to vectorize movie reviews. 

TextVectorization layer is a text tokenizer which breaks up the text into words (it is similar to Keras Tokenizer but implemented as a layer). You can read more about TextVectorization layer [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization).


In [ ]:
# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 200
# Use the text vectorization layer to normalize, split, and map strings to 
# integers.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE, 
    output_sequence_length=MAX_SEQUENCE_LENGTH
)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [ ]:
print(len(vectorize_layer.get_vocabulary()))

## Create a classification model

<img src="https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/resources/it3103/bidirectionalRNN.png"/>

Above is a diagram of the model. 

1. This model can be built as a `tf.keras.Sequential`.

2. The first layer is the vectorization layer, which converts the text to a sequence of token indices.

3. After the vectorization layer is an embedding layer. An embedding layer stores one vector per word. When called, it converts the sequences of word indices to sequences of vectors. These vectors are trainable. After training (on enough data), words with similar meanings often have similar vectors.

  This index-lookup is much more efficient than the equivalent operation of passing a one-hot encoded vector through a `tf.keras.layers.Dense` layer.

4. A recurrent neural network (RNN) processes sequence input by iterating through the elements. RNNs pass the outputs from one timestep to their input on the next timestep.

  The `tf.keras.layers.Bidirectional` wrapper can also be used with an RNN layer. This propagates the input forward and backwards through the RNN layer and then concatenates the final output. 

  * The main advantage of a bidirectional RNN is that the signal from the beginning of the input doesn't need to be processed all the way through every timestep to affect the output.  

  * The main disadvantage of a bidirectional RNN is that you can't efficiently stream predictions as words are being added to the end.

5. After the RNN has converted the sequence to a single vector the two `layers.Dense` do some final processing, and convert from this vector representation to a single logit as the classification output. 

In [ ]:
EMBEDDING_DIM=128

model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, 
              output_dim=EMBEDDING_DIM, 
              mask_zero=True, 
              name='embedding'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

## Compile and train the model

You will use [TensorBoard](https://www.tensorflow.org/tensorboard) to visualize metrics including loss and accuracy. Create a `tf.keras.callbacks.TensorBoard`.

In [ ]:
root_logdir = os.path.join(os.curdir, "tb_logs")

def get_run_logdir():    # use a new directory for each run
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=run_logdir)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="bestcheckpoint",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

Compile and train the model using the `Adam` optimizer and `BinaryCrossentropy` loss. 

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])


In [ ]:
model.fit(
    train_ds, 
    validation_data=val_ds,
    epochs=5, 
    callbacks=[tensorboard_callback, model_checkpoint_callback])

Visualize the model metrics in TensorBoard.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs

The model reaches a validation accuracy of around 85% after 1 epoch of training.

Note: Your results may be a bit different, depending on how weights were randomly initialized before training the embedding layer. 


Let's evaluate the model on our test dataset.

In [ ]:
test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=128)

In [ ]:
model.evaluate(test_ds)

Here we show how we can use get all the individual predictions for the test_ds and use the predictions to plot the confusion_matrix and classification report to allow us to have better insight.

In [ ]:
import numpy as np

y_preds = np.array([])
y_labels = np.array([])
count = 0
for texts, labels in test_ds:
    preds = model.predict(texts)
    preds = (preds >= 0.5).reshape(-1)
    y_preds = np.concatenate((y_preds, preds), axis=0)
    y_labels = np.concatenate((y_labels, labels), axis=0)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_labels, y_preds))


Let's go ahead and save our model. You will see that our model achieve an accuracy of around 82%. 

In [ ]:
model.save('sentiment_model')

Now let us put our model in use!!  We will first load our saved model.


In [ ]:
loaded_model = tf.keras.models.load_model('sentiment_model')

In [ ]:
loaded_model.summary()

Run the following cell and type in your own text at the prompt:

In [ ]:
text = input("Write your review here:")

In [ ]:
pred = loaded_model.predict([text])[0]
if pred >= 0.5: 
    print('positive sentiment')
else:
    print('negative sentiment')

## Stack two or more LSTM layers

Keras recurrent layers have two available modes that are controlled by the `return_sequences` constructor argument:

* If `False` it returns only the last output for each input sequence (a 2D tensor of shape (batch_size, output_features)). This is the default, used in the previous model.

* If `True` the full sequences of successive outputs for each timestep is returned (a 3D tensor of shape `(batch_size, timesteps, output_features)`).

Here is what the flow of information looks like with `return_sequences=True`:

<img src="https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/resources/it3103/layered_bidirectional.png"/>

In [ ]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
model.fit(train_ds, 
          epochs=5,
          validation_data=val_ds,
          callbacks=[tensorboard_callback, model_checkpoint_callback])

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
sample_text = 'i dozed off during the show.'
predictions = model.predict(np.array([sample_text]))
print( 'positive' if predictions >= 0.5 else 'negative')

## Exercises

Experiment with any of following to see if you get better or worse validation accuracy.

1. Add in Dropout layer
2. Increase vocabulary size 
2. Increase Embedding dimensions 
4. Use uni-directional LSTM instead of bidirectional
